<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [4]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [5]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [12]:
# import nlp libraries and modules
import spacy
from spacy.tokenizer import Tokenizer

# create instance of nlp engine
nlp = spacy.load("en_core_web_lg")

# insantiate tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [10]:
# using a sample for quick results while building function
sample = 'Friends, Romans, countrymen, lend me your ears;'

In [18]:
# tokenizing sample
list_sample = [token.text for token in tokenizer(sample)]

In [22]:
# convert to df for function testing
df = pd.DataFrame(list_sample)
df = df.rename(columns={0: 'text'})

In [57]:
def tokenize(doc):
    """Parses a string into words      
    Args:
        doc (text feature): turns text feature into tokens
    Returns:
        Each observation tokenized"""
    
    tokens = []
    
    for observation in tokenizer.pipe(doc, batch_size=500):
        
        doc_tokens = []
        
        # Remove stop words and punctuation
        for token in observation:
            if(token.is_stop == False) and (token.is_punct == False):
                doc_tokens.append(token.text.lower())        
        
        tokens.append(doc_tokens)
        
    return tokens

In [59]:
# apply function
yelp['tokens'] = tokenize(yelp['text'])

In [60]:
# view results
yelp['tokens'].head(10)

0    [beware!!!, fake,, fake,, fake....we, small, b...
1    [came, lunch, togo., service, quick., staff, f...
2    [i've, vegas, dozens, times, stepped, foot, ci...
3    [went, night, closed, street, party..., best, ...
4    [3.5, 4, stars, \n\n, bad, price,, $12.99, lun...
5    [tasty,, fast, casual, latin, street, food.,  ...
6    [absolutely, amazing!!, incredible, production...
7    [came, pho, enjoyed, it!,  , got, 9:00pm, busy...
8    [absolutely, unique, experience, nail, shop, f...
9    [wow., walked, sat, bar, 10, minutes., bartend...
Name: tokens, dtype: object

In [53]:
def get_lemmas(text):
    
    lemmas = []
    
    doc = nlp(text)
    
    for token in doc:
        if((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

In [54]:
yelp['lemmas'] = yelp['text'].apply(get_lemmas)

In [56]:
yelp['lemmas'].head()

0    [beware, fake, fake, fake, small, business, Lo...
1    [come, lunch, Togo, service, quick, staff, fri...
2    [Vegas, dozen, time, step, foot, Circus, Circu...
3    [go, night, close, street, party, good, actual...
4    [3.5, 4, star, \n\n, bad, price, $, 12.99, lun...
Name: lemmas, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [62]:
''' Import Statements'''

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

In [64]:
# instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english',
                         ngram_range=(1,2),
                         max_df=.97,
                         min_df=3,
                         tokenizer=get_lemmas)

# create a vocabulary and get word counts
dtm = tfidf.fit_transform(yelp['text'])

# get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# view Feature Matrix as DataFrame
dtm.head()

,,,,,,,,,$,1,...,à,à ce,à essayer,à la,ça,équipe,était,été,être,‍
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.027527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.155970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
dtm.shape

(10000, 33512)

In [75]:
# fit NearestNeighbors on DTM
nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [68]:
# create fake review
fake_review = ["I found the restaraunt to be very nice, although the servers were a bit annoying"]

In [69]:
# add to model
fake_tfidf = tfidf.transform(fake_review)

In [108]:
fake_tfidf

<1x33512 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [107]:
results = nn.kneighbors(fake_tfidf.todense())

In [101]:
# make a cleaner list
similar_reviews = results[1]

for review in similar_reviews:
    matches = pd.DataFrame(yelp['text'][review)

In [105]:
matches

,text
6311,天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用...
9333,I had tried their weekday buffet and it was mo...
5138,Love the quick appetizers! Great food and good...
8919,This is a very good hair salon but is a little...
8365,First of a let me start off by saying I love t...
8085,You can't really NOT go to a restaurant with a...
8074,"Looking for somewhere different to go, we deci..."
2208,"Bad service, if you like an unopened bottle of..."
5561,"Came here for a friends bday dinner, and the i..."
2420,After multiple times passing by and not been a...


It seems like all of these reviews start positive but also include something negative

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [125]:
# imports
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [117]:
yelp.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id', 'tokens', 'lemmas'],
      dtype='object')

In [139]:
# split up data
target = 'stars'
X = yelp['text']
y = yelp[target]

# train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                    test_size=0.2, random_state=11, stratify=y)

In [133]:
X_train.shape, y_train.shape

((8000,), (8000,))

In [134]:
# create instance of RandomForestClassifier and Vectorizer
rfc = RandomForestClassifier()

vect = TfidfVectorizer(stop_words='english')

In [135]:
# define the pipeline
pipe = Pipeline([
                ('vect', vect),
                ('clf', rfc)
                ])

In [147]:
# set parameters
parameters = {'vect__max_df': (0.75, 1.0),
              'vect__min_df': (.02, .05),
              'vect__max_features': (500, 1000),
              'clf__n_estimators':(5, 10,),
              'clf__max_depth':(15, 20)}

# run pipeline
grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
# fit
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [148]:
# check score (yikes)
grid_search.best_score_

0.541375

In [151]:
pred_rating = grid_search.predict(fake_review)

In [153]:
print('Predicted Rating of Review:', pred_rating[0],'Stars')

Predicted Rating of Review: 5 Stars


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [154]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

C:\Users\Sl0thstronaught\Anaconda3\envs\u4-s1-nlp\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Learn the vocubalary of the yelp data:

In [157]:
id2word = Dictionary(yelp['lemmas'])

Create a bag of words representation of the entire corpus

In [159]:
corpus = [id2word.doc2bow(text) for text in yelp['lemmas']]

In [160]:
corpus[0][:10]

[(0, 4),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

Your LDA model should be ready for estimation: 

In [161]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

In [162]:
lda.print_topics()

[(0,
  '0.027*" " + 0.019*"\n\n" + 0.015*"good" + 0.009*"place" + 0.008*"go" + 0.008*"time" + 0.008*"come" + 0.007*"order" + 0.007*"like" + 0.006*"food"'),
 (1,
  '0.030*" " + 0.015*"\n\n" + 0.011*"good" + 0.010*"great" + 0.009*"time" + 0.009*"food" + 0.009*"come" + 0.008*"like" + 0.007*"place" + 0.006*"service"'),
 (2,
  '0.030*" " + 0.015*"\n\n" + 0.014*"good" + 0.010*"place" + 0.008*"come" + 0.008*"great" + 0.008*"time" + 0.007*"food" + 0.007*"like" + 0.006*"go"'),
 (3,
  '0.039*" " + 0.014*"\n\n" + 0.012*"good" + 0.011*"food" + 0.009*"great" + 0.009*"time" + 0.009*"place" + 0.007*"service" + 0.007*"like" + 0.006*"come"'),
 (4,
  '0.028*" " + 0.013*"great" + 0.012*"place" + 0.011*"good" + 0.011*"food" + 0.010*"time" + 0.010*"\n\n" + 0.008*"get" + 0.008*"come" + 0.007*"like"'),
 (5,
  '0.015*" " + 0.014*"\n\n" + 0.009*"place" + 0.009*"good" + 0.008*"food" + 0.008*"like" + 0.007*"time" + 0.007*"come" + 0.007*"\n" + 0.007*"great"'),
 (6,
  '0.022*" " + 0.015*"good" + 0.012*"\n\n" + 0.0

Create 1-2 visualizations of the results

In [164]:
import re

words = [re.findall(r'"([^"]*)"', t[1]) for t in lda.print_topics()]

In [166]:
topics = [' '.join(t[0:5]) for t in words]

In [167]:
for id, t in enumerate(topics):
    print(f'------- Topic -------')
    print(t, end='\n\n')

------- Topic -------
  

 good place go

------- Topic -------
  

 good great time

------- Topic -------
  

 good place come

------- Topic -------
  

 good food great

------- Topic -------
  great place good food

------- Topic -------
  

 place good food

------- Topic -------
  good 

 
 place

------- Topic -------


   good place food

------- Topic -------
  

 good service food

------- Topic -------
  

 place food good



In [168]:
# use pyLDAvis for topic visualization
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [169]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\Sl0thstronaught\Anaconda3\envs\u4-s1-nlp\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.002298 -0.004152       1        1  10.634191
0      0.000375  0.005247       2        1  10.469356
9      0.001440  0.000888       3        1  10.301826
3      0.005405 -0.001626       4        1  10.211017
1      0.001912  0.000562       5        1  10.160639
7     -0.006083  0.000116       6        1  10.151143
2      0.001876  0.000546       7        1   9.865851
8     -0.000320  0.002369       8        1   9.767164
5     -0.005208 -0.002207       9        1   9.283624
6     -0.001693 -0.001745      10        1   9.155186, topic_info=    Category          Freq        Term         Total  loglift  logprob
0    Default  13757.000000              13757.000000  30.0000  30.0000
55   Default   8156.000000        \n\n   8156.000000  29.0000  29.0000
321  Default   3054.000000          \n   3054.000000  28.0000  28.0000
208  Default   4117.000000       great   4117.000000  27.0000  27.0000
46   Default   6132.000000        good   6132.000000  26.0000  26.0000
1    Default   2167.000000           $   2167.000000  25.0000  25.0000
374  Default   3289.000000       order   3289.000000  24.0000  24.0000
131  Default   5070.000000       place   5070.000000  23.0000  23.0000
83   Default   2963.000000          go   2963.000000  22.0000  22.0000
129  Default   1325.000000      people   1325.000000  21.0000  21.0000
146  Default   4684.000000        food   4684.000000  20.0000  20.0000
393  Default   1438.000000        take   1438.000000  19.0000  19.0000
319  Default   1465.000000        work   1465.000000  18.0000  18.0000
43   Default   3752.000000        come   3752.000000  17.0000  17.0000
156  Default   1692.000000       price   1692.000000  16.0000  16.0000
51   Default   3450.000000     service   3450.000000  15.0000  15.0000
39   Default   4122.000000        time   4122.000000  14.0000  14.0000
82   Default   2791.000000         get   2791.000000  13.0000  13.0000
379  Default    726.000000       pizza    726.000000  12.0000  12.0000
52   Default   1481.000000       staff   1481.000000  11.0000  11.0000
40   Default   2461.000000         try   2461.000000  10.0000  10.0000
497  Default   1468.000000        tell   1468.000000   9.0000   9.0000
266  Default   1412.000000  experience   1412.000000   8.0000   8.0000
107  Default   1087.000000        room   1087.000000   7.0000   7.0000
97   Default   1733.000000        nice   1733.000000   6.0000   6.0000
692  Default   2143.000000        love   2143.000000   5.0000   5.0000
64   Default    999.000000         bar    999.000000   4.0000   4.0000
45   Default   1348.000000    friendly   1348.000000   3.0000   3.0000
430  Default   1458.000000         day   1458.000000   2.0000   2.0000
96   Default   1324.000000        need   1324.000000   1.0000   1.0000
..       ...           ...         ...           ...      ...      ...
64   Topic10    112.013130         bar    999.462036   0.2022  -6.0536
242  Topic10     98.336548       enjoy    865.936584   0.2154  -6.1838
146  Topic10    420.493713        food   4684.767090  -0.0198  -4.7308
876  Topic10     96.137993       clean    846.359985   0.2157  -6.2064
0    Topic10   1047.936401              13757.323242  -0.1839  -3.8176
83   Topic10    276.810608          go   2963.053955   0.0202  -5.1489
887  Topic10    115.687569         lot   1063.165405   0.1727  -6.0213
457  Topic10    164.704880        know   1630.900757   0.0981  -5.6680
329  Topic10     82.124840         big    714.991150   0.2268  -6.3640
404  Topic10    109.695007           2   1010.975708   0.1699  -6.0745
24   Topic10    316.978912        like   3635.851318  -0.0489  -5.0134
133  Topic10    116.428070       table   1099.382812   0.1456  -6.0149
621  Topic10    138.268402       drink   1362.941528   0.1026  -5.8430
27   Topic10    179.907333        look   1910.386475   0.0282  -5.5798
55   Topic10    555.544800        \n\n   81

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)